In [1]:
# this just to make sure we are using only on CPU
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [2]:
%cd ..

/volatile/home/Zaccharie/workspace/Deep-MRI-Reconstruction


In [3]:
import os.path as op
import time

import torch
from torch.optim import Adam
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm_notebook

from cascadenet_pytorch.model_pytorch import DnCn
from data_torch import MaskedUntouched2DDataset, MaskedUntouched2DAllLoadedDataset
from torch_training import fit_torch, torch_psnr

/volatile/home/Zaccharie/workspace/Deep-MRI-Reconstruction/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/volatile/home/Zaccharie/workspace/Deep-MRI-Reconstruction/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/volatile/home/Zaccharie/workspace/Deep-MRI-Reconstruction/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [5]:
# data loader
# generators
AF = 4
# train_gen = MaskedUntouched2DAllLoadedDataset(train_path, af=AF, inner_slices=1)
train_gen = MaskedUntouched2DDataset(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DDataset(val_path, af=AF)
train_loader = DataLoader(
    dataset=train_gen,
    batch_size=1,
    shuffle=False,
    num_workers=35,
#     pin_memory=True,
)
val_loader = DataLoader(
    dataset=val_gen,
    batch_size=1,
    num_workers=35,
#     pin_memory=True,
)

In [6]:
run_params = {
    'nc': 5,  # n cascade
    'nd': 5,  # n convs
    'nf': 64,  # n filters
}
n_epochs = 100
run_id = f'cascadenet_orig_torch_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
log_dir = op.join('logs', run_id)
print(run_id)

cascadenet_orig_torch_af4_1567439027


In [7]:
model = DnCn(**run_params)
optimizer = Adam(model.parameters(), lr=1e-3)
writer = SummaryWriter(log_dir=log_dir)

# model.cuda();

Creating D5C5


In [8]:
def overfit_epoch(model, data, optimizer, device):
    model.train()
    kspace, mask, image_gt = data
    kspace = kspace[0]
    mask = mask[0]
    image_gt = image_gt[0]
    kspace = kspace.to(device)
    mask = mask.to(device)
    image_gt = image_gt.to(device)
    image_pred = model(kspace, mask)

    loss = F.l1_loss(image_pred, image_gt)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    psnr = torch_psnr(image_pred, image_gt)
    print('Training PSNR:', psnr)

In [ ]:
%%time
i, data = next(enumerate(train_loader))
for _ in tqdm_notebook(range(500)):
    overfit_epoch(model, data, optimizer, 'cpu')
#     overfit_epoch(model, data, optimizer, 'cuda')    

Training PSNR: tensor(2.6834, grad_fn=<MulBackward0>)
Training PSNR: tensor(-1.8862, grad_fn=<MulBackward0>)
Training PSNR: tensor(1.1830, grad_fn=<MulBackward0>)
Training PSNR: tensor(11.5652, grad_fn=<MulBackward0>)
Training PSNR: tensor(6.8393, grad_fn=<MulBackward0>)
Training PSNR: tensor(4.5380, grad_fn=<MulBackward0>)
Training PSNR: tensor(7.0447, grad_fn=<MulBackward0>)
Training PSNR: tensor(15.7987, grad_fn=<MulBackward0>)
Training PSNR: tensor(14.5460, grad_fn=<MulBackward0>)
Training PSNR: tensor(10.4893, grad_fn=<MulBackward0>)
Training PSNR: tensor(11.2694, grad_fn=<MulBackward0>)
Training PSNR: tensor(16.8111, grad_fn=<MulBackward0>)
Training PSNR: tensor(23.8209, grad_fn=<MulBackward0>)
Training PSNR: tensor(16.3975, grad_fn=<MulBackward0>)
Training PSNR: tensor(16.1583, grad_fn=<MulBackward0>)
Training PSNR: tensor(20.2643, grad_fn=<MulBackward0>)
Training PSNR: tensor(28.0665, grad_fn=<MulBackward0>)
Training PSNR: tensor(20.5738, grad_fn=<MulBackward0>)
Training PSNR: 

Training PSNR: tensor(35.6274, grad_fn=<MulBackward0>)
Training PSNR: tensor(35.1370, grad_fn=<MulBackward0>)
Training PSNR: tensor(38.1942, grad_fn=<MulBackward0>)
Training PSNR: tensor(33.5076, grad_fn=<MulBackward0>)
Training PSNR: tensor(36.9246, grad_fn=<MulBackward0>)
Training PSNR: tensor(34.4131, grad_fn=<MulBackward0>)
Training PSNR: tensor(36.0931, grad_fn=<MulBackward0>)
Training PSNR: tensor(36.1122, grad_fn=<MulBackward0>)
Training PSNR: tensor(34.3224, grad_fn=<MulBackward0>)
Training PSNR: tensor(37.6400, grad_fn=<MulBackward0>)
Training PSNR: tensor(34.1441, grad_fn=<MulBackward0>)
Training PSNR: tensor(38.7764, grad_fn=<MulBackward0>)
Training PSNR: tensor(34.6908, grad_fn=<MulBackward0>)
Training PSNR: tensor(37.9845, grad_fn=<MulBackward0>)
Training PSNR: tensor(35.1309, grad_fn=<MulBackward0>)
Training PSNR: tensor(35.5988, grad_fn=<MulBackward0>)
Training PSNR: tensor(36.7908, grad_fn=<MulBackward0>)
Training PSNR: tensor(35.2158, grad_fn=<MulBackward0>)
Training P